# Notebook de treinamento
Bibliotecas

In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd

import velha as jv

from collections import OrderedDict

In [2]:
versao = "-v0.0"
sobrescreve = False

In [3]:
niveis_treinamento = OrderedDict({})
labels = ['muito fraco', 'fraco', 'medio', 'forte', 'muito forte']
menor_potencia = 2
maior_potencia = menor_potencia + len(labels)
rodadas = 10 ** np.arange(menor_potencia, maior_potencia)
for indice, label in enumerate(labels):
    niveis_treinamento[label] = {'rodadas': rodadas[indice]}
    niveis_treinamento[label]['progresso'] = widgets.FloatProgress(
        value=0.0,
        min=0.0,
        max=1.0,
        description=f'{label}:',
        bar_style='success',
        style={'bar_color': 'black'},
        orientation='horizontal'
    )

In [4]:
# Treinamento de dois jogadores por um número de rodadas

for nivel in niveis_treinamento:
    rodadas_treinamento = niveis_treinamento[nivel]['rodadas']
    print(f'Iniciando treinamento com {rodadas_treinamento} rodadas... ({nivel})')
    verifica = rodadas_treinamento / 10
    politica_X = f"X{rodadas_treinamento}{versao}"
    politica_O = f"O{rodadas_treinamento}{versao}"

    if sobrescreve or (not jv.existe_politica(politica_X) or not jv.existe_politica(politica_O)):
        primeiro = jv.Maquina(politica_X)
        segundo = jv.Maquina(politica_O)
        treino = jv.jogoDaVelha(primeiro, segundo)
        treino.treinamento(rodadas_treinamento, verifica=verifica, progresso=niveis_treinamento[nivel]['progresso'])
        print(f"Salvando políticas para {rodadas_treinamento} rodadas")
        print(f'Salvando política: {primeiro.nome}')
        primeiro.salva_politica()
        print(f'Salvando política: {segundo.nome}')
        segundo.salva_politica()
    else:
        print(f'Políticas {politica_O} e {politica_X} já existem')

Iniciando treinamento com 100 rodadas... (muito fraco)


FloatProgress(value=0.0, bar_style='success', description='muito fraco:', max=1.0, style=ProgressStyle(bar_col…

Treinamento finalizado: 100 rodadas
Salvando políticas para 100 rodadas
Salvando política: X100-v0.0
Salvando política: O100-v0.0
Iniciando treinamento com 1000 rodadas... (fraco)


FloatProgress(value=0.0, bar_style='success', description='fraco:', max=1.0, style=ProgressStyle(bar_color='bl…

Treinamento finalizado: 1000 rodadas
Salvando políticas para 1000 rodadas
Salvando política: X1000-v0.0
Salvando política: O1000-v0.0
Iniciando treinamento com 10000 rodadas... (medio)


...

Treinamento finalizado: 10000 rodadas
Salvando políticas para 10000 rodadas
Salvando política: X10000-v0.0
Salvando política: O10000-v0.0
Iniciando treinamento com 100000 rodadas... (forte)


FloatProgress(value=0.0, bar_style='success', description='forte:', max=1.0, style=ProgressStyle(bar_color='bl…

Treinamento finalizado: 100000 rodadas
Salvando políticas para 100000 rodadas
Salvando política: X100000-v0.0
Salvando política: O100000-v0.0
Iniciando treinamento com 1000000 rodadas... (muito forte)


FloatProgress(value=0.0, bar_style='success', description='muito forte:', max=1.0, style=ProgressStyle(bar_col…

Treinamento finalizado: 1000000 rodadas
Salvando políticas para 1000000 rodadas
Salvando política: X1000000-v0.0
Salvando política: O1000000-v0.0


# Combinando políticas

In [5]:
for nivel in niveis_treinamento:
    politica_combinada = f"XO{niveis_treinamento[nivel]['rodadas']}{versao}"
    if sobrescreve or not jv.existe_politica(politica_combinada):
        primeiro = jv.Maquina("velhaRLX", taxa_exploracao=0.0)
        primeiro.carrega_politica(f"X{niveis_treinamento[nivel]['rodadas']}{versao}")
        segundo = jv.Maquina("velhaRLO", taxa_exploracao=0.0)
        segundo.carrega_politica(f"O{niveis_treinamento[nivel]['rodadas']}{versao}")

        primeiro.combina_e_salva_politica(segundo, politica_combinada)
        print(f"Salvando política {politica_combinada}!!!")
    else:
        print(f"Arquivo de política {politica_combinada} já existe!")

Salvando política XO100-v0.0!!!
Salvando política XO1000-v0.0!!!
Salvando política XO10000-v0.0!!!
Salvando política XO100000-v0.0!!!
Salvando política XO1000000-v0.0!!!


# Jogando todas políticas contra todas

In [6]:
num_partidas = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step = 100,
    description='Rodadas: '
)
num_partidas

IntSlider(value=10000, description='Rodadas: ', max=50000, min=1000, step=100)

In [7]:
resultados = {}
for nivel_X in niveis_treinamento:
    resultados['X ' + nivel_X] = {}
    for nivel_O in niveis_treinamento:
        rodadas_X = niveis_treinamento[nivel_X]['rodadas']
        rodadas_O = niveis_treinamento[nivel_O]['rodadas']
        X = jv.Maquina(f"X-{rodadas_X}", taxa_exploracao=0.0, limite_exploracao=0.0)
        X.carrega_politica(f"X{rodadas_X}{versao}")
        O = jv.Maquina(f"O-{rodadas_O}", taxa_exploracao=0.0, limite_exploracao=0.0)
        O.carrega_politica(f"O{rodadas_O}{versao}")

        simulacao = jv.jogoDaVelha(X, O)
        total, tabuleiros = simulacao.simulacao(partidas=num_partidas.value)
        resultados['X ' + nivel_X]['O ' + nivel_O] = total
        print(f"{X.nome}, {O.nome}")
        print(total)
        
        print()

X-100, O-100
Counter({'O-100': 4017, 'X-100': 3738, 'Velha': 2245})

X-100, O-1000
Counter({'Velha': 5960, 'X-100': 2374, 'O-1000': 1666})

X-100, O-10000
Counter({'Velha': 5340, 'O-10000': 4660})

X-100, O-100000
Counter({'O-100000': 10000})

X-100, O-1000000
Counter({'O-1000000': 10000})

X-1000, O-100
Counter({'X-1000': 5869, 'O-100': 3850, 'Velha': 281})

X-1000, O-1000
Counter({'O-1000': 6702, 'Velha': 2470, 'X-1000': 828})

X-1000, O-10000
Counter({'O-10000': 8316, 'Velha': 1684})

X-1000, O-100000
Counter({'O-100000': 7512, 'Velha': 2488})

X-1000, O-1000000
Counter({'O-1000000': 7490, 'Velha': 2510})

X-10000, O-100
Counter({'X-10000': 7968, 'O-100': 2032})

X-10000, O-1000
Counter({'X-10000': 10000})

X-10000, O-10000
Counter({'Velha': 10000})

X-10000, O-100000
Counter({'O-100000': 10000})

X-10000, O-1000000
Counter({'O-1000000': 10000})

X-100000, O-100
Counter({'X-100000': 10000})

X-100000, O-1000
Counter({'X-100000': 10000})

X-100000, O-10000
Counter({'Velha': 10000})



In [8]:
pd.DataFrame(resultados)

,X muito fraco,X fraco,X medio,X forte,X muito forte
O muito fraco,"{'O-100': 4017, 'X-100': 3738, 'Velha': 2245}","{'O-100': 3850, 'X-1000': 5869, 'Velha': 281}","{'O-100': 2032, 'X-10000': 7968}",{'X-100000': 10000},"{'X-1000000': 8772, 'Velha': 1228}"
O fraco,"{'O-1000': 1666, 'Velha': 5960, 'X-100': 2374}","{'O-1000': 6702, 'Velha': 2470, 'X-1000': 828}",{'X-10000': 10000},{'X-100000': 10000},{'Velha': 10000}
O medio,"{'O-10000': 4660, 'Velha': 5340}","{'O-10000': 8316, 'Velha': 1684}",{'Velha': 10000},{'Velha': 10000},{'Velha': 10000}
O forte,{'O-100000': 10000},"{'O-100000': 7512, 'Velha': 2488}",{'O-100000': 10000},{'Velha': 10000},{'Velha': 10000}
O muito forte,{'O-1000000': 10000},"{'O-1000000': 7490, 'Velha': 2510}",{'O-1000000': 10000},{'Velha': 10000},{'Velha': 10000}
